# Wan2.2 Video Generation on Google Colab

Wan2.2をGoogle Colabで動かすためのノートブック

## 特徴
- GPU自動検出と最適化設定
- ComfyUI版と公式CLI版の両方に対応
- T4 (15GB) / A100 (40GB) 対応
- メモリ最適化オプション

## 対応モデル
| モデル | タイプ | VRAM | T4対応 |
|--------|--------|------|--------|
| TI2V-5B | テキスト＆画像→動画 | 8GB | OK |
| I2V-14B | 画像→動画 | 24GB+ | 要量子化 |
| T2V-14B | テキスト→動画 | 24GB+ | 要量子化 |

---
## 1. 環境確認とGPU検出

In [1]:
import subprocess
import torch

def get_gpu_info():
    """GPU情報を取得して最適な設定を返す"""
    if not torch.cuda.is_available():
        return None, 0, {}
    
    gpu_name = torch.cuda.get_device_name(0)
    gpu_memory = torch.cuda.get_device_properties(0).total_memory / (1024**3)
    
    # GPU別の推奨設定
    if "A100" in gpu_name:
        config = {
            "recommended_model": "14B",
            "max_resolution": "1280x720",
            "offload_model": False,
            "t5_cpu": False,
            "use_fp8": True
        }
    elif "T4" in gpu_name or gpu_memory < 20:
        config = {
            "recommended_model": "5B",
            "max_resolution": "832x480",
            "offload_model": True,
            "t5_cpu": True,
            "use_fp8": True
        }
    else:
        config = {
            "recommended_model": "14B",
            "max_resolution": "1280x720",
            "offload_model": True,
            "t5_cpu": False,
            "use_fp8": True
        }
    
    return gpu_name, gpu_memory, config

# GPU情報を表示
!nvidia-smi
print("\n" + "="*50)
gpu_name, gpu_memory, gpu_config = get_gpu_info()
print(f"GPU: {gpu_name}")
print(f"VRAM: {gpu_memory:.1f} GB")
print(f"推奨モデル: {gpu_config.get('recommended_model', 'N/A')}")
print(f"推奨解像度: {gpu_config.get('max_resolution', 'N/A')}")
print("="*50)

Mon Dec 29 16:06:07 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 550.54.15              Driver Version: 550.54.15      CUDA Version: 12.4     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  Tesla T4                       Off |   00000000:00:04.0 Off |                    0 |
| N/A   42C    P8              9W /   70W |       0MiB /  15360MiB |      0%      Default |
|                                         |                        |                  N/A |
+-----------------------------------------+-----

---
## 2. 設定

In [ ]:
#@title 設定パネル { display-mode: "form" }

#@markdown ### 実行モード
EXECUTION_MODE = "comfyui" #@param ["comfyui", "cli"]

#@markdown ### モデル選択
MODEL_TYPE = "TI2V-5B" #@param ["TI2V-5B", "I2V-14B", "T2V-14B"]

#@markdown ### 解像度
RESOLUTION = "832x480" #@param ["1280x720", "832x480", "480x832", "1024x1024"]

#@markdown ### メモリ最適化（T4では全てON推奨）
OFFLOAD_MODEL = True #@param {type:"boolean"}
T5_ON_CPU = True #@param {type:"boolean"}
CONVERT_DTYPE = True #@param {type:"boolean"}

#@markdown ### Google Drive連携
USE_GOOGLE_DRIVE = False #@param {type:"boolean"}
DRIVE_OUTPUT_DIR = "/content/drive/MyDrive/Wan2.2_outputs" #@param {type:"string"}

#@markdown ### トンネル設定（ComfyUIモード用）
NGROK_TOKEN = "" #@param {type:"string"}

# 設定の表示
print("=== 現在の設定 ===")
print(f"実行モード: {EXECUTION_MODE}")
print(f"モデル: {MODEL_TYPE}")
print(f"解像度: {RESOLUTION}")
print(f"メモリ最適化: offload={OFFLOAD_MODEL}, t5_cpu={T5_ON_CPU}, convert={CONVERT_DTYPE}")

---
## 3. Google Drive連携（オプション）

In [ ]:
import os

if USE_GOOGLE_DRIVE:
    from google.colab import drive
    drive.mount('/content/drive')
    os.makedirs(DRIVE_OUTPUT_DIR, exist_ok=True)
    print(f"Google Drive connected. Output: {DRIVE_OUTPUT_DIR}")
else:
    print("Google Drive: 未使用")

---
## 4. インストール

In [ ]:
%%time
import os

if EXECUTION_MODE == "comfyui":
    # ComfyUIのインストール
    if not os.path.exists("/content/ComfyUI"):
        !git clone https://github.com/comfyanonymous/ComfyUI.git /content/ComfyUI
    %cd /content/ComfyUI
    !pip install -q -r requirements.txt
    !pip install -q huggingface_hub
    print("ComfyUI installed.")
else:
    # 公式Wan2.2のインストール
    if not os.path.exists("/content/Wan2.2"):
        !git clone https://github.com/Wan-Video/Wan2.2.git /content/Wan2.2
    %cd /content/Wan2.2
    !pip install -q -r requirements.txt
    !pip install -q .
    !pip install -q huggingface_hub
    print("Wan2.2 CLI installed.")

---
## 5. モデルダウンロード

In [ ]:
%%time
from huggingface_hub import hf_hub_download, snapshot_download
import os

if EXECUTION_MODE == "comfyui":
    # ComfyUI用モデルディレクトリ
    os.makedirs("models/diffusion_models", exist_ok=True)
    os.makedirs("models/clip", exist_ok=True)
    os.makedirs("models/vae", exist_ok=True)
    
    print(f"Downloading {MODEL_TYPE} for ComfyUI...")
    
    # Diffusion Model
    if MODEL_TYPE == "TI2V-5B":
        hf_hub_download(
            repo_id="Comfy-Org/Wan_2.2_ComfyUI_repackaged",
            filename="split_files/diffusion_models/wan2.2_ti2v_5B_fp16.safetensors",
            local_dir="models"
        )
    elif MODEL_TYPE == "I2V-14B":
        hf_hub_download(
            repo_id="Comfy-Org/Wan_2.2_ComfyUI_repackaged",
            filename="split_files/diffusion_models/wan2.2_i2v_A14B_fp16_high_noise.safetensors",
            local_dir="models"
        )
        hf_hub_download(
            repo_id="Comfy-Org/Wan_2.2_ComfyUI_repackaged",
            filename="split_files/diffusion_models/wan2.2_i2v_A14B_fp16_low_noise.safetensors",
            local_dir="models"
        )
    elif MODEL_TYPE == "T2V-14B":
        hf_hub_download(
            repo_id="Comfy-Org/Wan_2.2_ComfyUI_repackaged",
            filename="split_files/diffusion_models/wan2.2_t2v_A14B_fp16.safetensors",
            local_dir="models"
        )
    
    # Text Encoder
    hf_hub_download(
        repo_id="Comfy-Org/Wan_2.2_ComfyUI_repackaged",
        filename="split_files/text_encoders/umt5_xxl_fp8_e4m3fn_scaled.safetensors",
        local_dir="models"
    )
    
    # VAE
    hf_hub_download(
        repo_id="Comfy-Org/Wan_2.2_ComfyUI_repackaged",
        filename="split_files/vae/wan_2.2_vae.safetensors",
        local_dir="models"
    )
    
    # ファイルを正しい場所に移動
    !mv models/split_files/diffusion_models/* models/diffusion_models/ 2>/dev/null || true
    !mv models/split_files/text_encoders/* models/clip/ 2>/dev/null || true
    !mv models/split_files/vae/* models/vae/ 2>/dev/null || true
    !rm -rf models/split_files 2>/dev/null || true
    
else:
    # 公式CLI用モデル
    print(f"Downloading {MODEL_TYPE} for CLI...")
    
    if MODEL_TYPE == "TI2V-5B":
        repo_id = "Wan-AI/Wan2.2-TI2V-5B"
    elif MODEL_TYPE == "I2V-14B":
        repo_id = "Wan-AI/Wan2.2-I2V-A14B"
    elif MODEL_TYPE == "T2V-14B":
        repo_id = "Wan-AI/Wan2.2-T2V-A14B"
    
    model_dir = f"./models/{MODEL_TYPE}"
    !huggingface-cli download {repo_id} --local-dir {model_dir}

print("\nModel download complete!")
!ls -la models/

---
## 6A. ComfyUIモードで実行

In [ ]:
if EXECUTION_MODE == "comfyui":
    # ngrokトンネル設定
    if NGROK_TOKEN:
        !pip install -q pyngrok
        from pyngrok import ngrok
        ngrok.set_auth_token(NGROK_TOKEN)
        public_url = ngrok.connect(8188)
        print(f"\n{'='*50}")
        print(f"ComfyUI URL: {public_url}")
        print(f"{'='*50}\n")
    else:
        print("ngrok未設定。localhostのみアクセス可能。")
        print("Colabの場合、出力ポートからアクセスしてください。")
    
    # ComfyUI起動
    !python main.py --listen 0.0.0.0 --port 8188
else:
    print("CLIモードが選択されています。次のセルに進んでください。")

---
## 6B. CLIモードで実行

In [ ]:
#@title 動画生成設定 { display-mode: "form" }

if EXECUTION_MODE == "cli":
    #@markdown ### 入力画像（I2Vモード用）
    INPUT_IMAGE = "/content/input.jpg" #@param {type:"string"}
    
    #@markdown ### プロンプト
    PROMPT = "A beautiful sunset over the ocean, waves gently rolling, cinematic quality" #@param {type:"string"}
    
    #@markdown ### フレーム数
    NUM_FRAMES = 49 #@param {type:"slider", min:17, max:121, step:4}
    
    print("設定完了。次のセルで動画生成を実行します。")
else:
    print("ComfyUIモードが選択されています。6Aセルを実行してください。")

In [ ]:
if EXECUTION_MODE == "cli":
    import torch
    import os
    
    # VRAMクリア
    torch.cuda.empty_cache()
    
    # タスク設定
    if MODEL_TYPE == "TI2V-5B":
        task = "ti2v-5B"
        ckpt_dir = "./models/TI2V-5B"
    elif MODEL_TYPE == "I2V-14B":
        task = "i2v-A14B"
        ckpt_dir = "./models/I2V-14B"
    elif MODEL_TYPE == "T2V-14B":
        task = "t2v-A14B"
        ckpt_dir = "./models/T2V-14B"
    
    # 解像度パース
    width, height = RESOLUTION.split("x")
    
    # 最適化オプション構築
    args = []
    if OFFLOAD_MODEL:
        args.append("--offload_model True")
    if T5_ON_CPU:
        args.append("--t5_cpu")
    if CONVERT_DTYPE:
        args.append("--convert_model_dtype")
    
    args_str = " ".join(args)
    
    # 出力ディレクトリ
    output_dir = DRIVE_OUTPUT_DIR if USE_GOOGLE_DRIVE else "./outputs"
    os.makedirs(output_dir, exist_ok=True)
    
    # 実行コマンド構築
    cmd = f"""python generate.py \
        --task {task} \
        --size {width}*{height} \
        --ckpt_dir {ckpt_dir} \
        --prompt "{PROMPT}" \
        --frame_num {NUM_FRAMES} \
        --save_path {output_dir} \
        {args_str}"""
    
    # 画像入力がある場合
    if "i2v" in task.lower() or "ti2v" in task.lower():
        if os.path.exists(INPUT_IMAGE):
            cmd += f' --image "{INPUT_IMAGE}"'
    
    print("実行コマンド:")
    print(cmd)
    print("\n" + "="*50)
    print("動画生成を開始します...")
    print("="*50 + "\n")
    
    !{cmd}

In [ ]:
# 生成された動画を表示
if EXECUTION_MODE == "cli":
    import glob
    from IPython.display import Video, display
    
    output_dir = DRIVE_OUTPUT_DIR if USE_GOOGLE_DRIVE else "./outputs"
    videos = sorted(glob.glob(f"{output_dir}/*.mp4"), key=os.path.getmtime, reverse=True)
    
    if videos:
        latest_video = videos[0]
        print(f"最新の動画: {latest_video}")
        display(Video(latest_video, embed=True, width=640))
    else:
        print("動画が見つかりません。")

---
## 7. 画像アップロード（I2V用）

In [ ]:
from google.colab import files
from IPython.display import Image, display
import shutil

print("画像をアップロードしてください...")
uploaded = files.upload()

if uploaded:
    for filename in uploaded.keys():
        # 入力画像として保存
        shutil.copy(filename, "/content/input.jpg")
        print(f"\n{filename} を /content/input.jpg として保存しました")
        display(Image(filename, width=400))

---
## Tips

### T4 GPU (15GB) での推奨設定
- モデル: TI2V-5B
- 解像度: 832x480
- 全メモリ最適化オプションON

### A100 GPU (40GB) での推奨設定
- モデル: I2V-14B または T2V-14B
- 解像度: 1280x720
- メモリ最適化は必要に応じて

### 生成時間目安
- T4 + 5B: 15-25分/動画
- A100 + 14B: 5-10分/動画

### トラブルシューティング
- OOMエラー: 解像度を下げる、メモリ最適化オプションをONに
- セッション切断: Google Drive連携で出力を保存